In [ ]:
#Import Libraries
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import numpy as np
import matplotlib.pyplot as plt
import cv2
from pathlib import Path
import os
import json
from tqdm import tqdm

# We set seed for reproducibility
torch.manual_seed(42)
# Reproducibility for NumPy random operations 
np.random.seed(42)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Used for debugging - shows if GPU is active          
print(f"Using device: {device}")

Using device: cpu


In [9]:
# Load pre-trained ResNet-18 with ImageNet weights
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
model = model.to(device)
model.eval()  # Set to evaluation mode

print("✓ ResNet-18 loaded successfully")
print(f"Model moved to device: {device}")

✓ ResNet-18 loaded successfully
Model moved to device: cpu


In [13]:
# Define CIFAR-10 normalization (ImageNet stats)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Load CIFAR-10 test set
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"✓ CIFAR-10 test set loaded: {len(test_dataset)} images")
print(f"CIFAR-10 classes: {test_dataset.classes}")

✓ CIFAR-10 test set loaded: 10000 images
CIFAR-10 classes: ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


/home/steven/Desktop/lab1-ai/.venv/lib/python3.13/site-packages/torchvision/datasets/cifar.py:83: VisibleDeprecationWarning: dtype(): align should be passed as Python or NumPy boolean but got `align=0`. Did you mean to pass a tuple to create a subarray type? (Deprecated NumPy 2.4)
  entry = pickle.load(f, encoding="latin1")


In [14]:
def evaluate_model_accuracy(model, data_loader, device, max_samples=1000):
    """Evaluate model accuracy on clean images"""
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in data_loader:
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted.cpu() == labels).sum().item()
            
            if total >= max_samples:
                break
    
    accuracy = (correct / total) * 100
    return accuracy, correct, total

# Test baseline accuracy
baseline_accuracy, correct, total = evaluate_model_accuracy(model, test_loader, device, max_samples=1000)
print(f"✓ Baseline accuracy on {total} clean CIFAR-10 images: {baseline_accuracy:.2f}%")

✓ Baseline accuracy on 1024 clean CIFAR-10 images: 0.00%


In [ ]:
def apply_pixel_change(image_tensor, pixel_x, pixel_y, r, g, b, device):
    """
    Apply a pixel change to an image tensor.
    
    Args:
        image_tensor: Original image tensor (normalized, shape [3, 32, 32])
        pixel_x: X coordinate (0-31)
        pixel_y: Y coordinate (0-31)
        r, g, b: RGB values in range [0, 1] (normalized for model input)
        device: torch device
    
    Returns:
        Modified image tensor
    """
    modified_image = image_tensor.clone()
    # Normalize pixel values to [-2, 2] range (ImageNet normalization inverse)
    # R channel
    modified_image[0, pixel_y, pixel_x] = (r - 0.485) / 0.229
    # G channel
    modified_image[1, pixel_y, pixel_x] = (g - 0.456) / 0.224
    # B channel
    modified_image[2, pixel_y, pixel_x] = (b - 0.426) / 0.225
    
    return modified_image

def predict_class(model, image_tensor, device):
    """
    Get model prediction and confidence for an image.
    
    Args:
        model: PyTorch model
        image_tensor: Image tensor (shape [3, 32, 32])
        device: torch device
    
    Returns:
        predicted_class, confidence scores
    """
    with torch.no_grad():
        image_batch = image_tensor.unsqueeze(0).to(device)
        outputs = model(image_batch)
        probabilities = torch.softmax(outputs, dim=1)
        predicted_class = torch.argmax(outputs, dim=1).item()
    
    return predicted_class, probabilities.cpu().numpy()[0]

def fitness_function(model, image_tensor, original_class, device):
    """
    Calculate fitness score for attack (higher = better attack).
    We want to maximize confidence of a WRONG class.
    
    Args:
        model: PyTorch model
        image_tensor: Modified image tensor
        original_class: True class label
        device: torch device

            Returns:
        fitness_score (float)
    """
    predicted_class, probabilities = predict_class(model, image_tensor, device)
    
    # If prediction is correct, fitness is low
    if predicted_class == original_class:
        return 0.0
    
    # If prediction is wrong, return confidence of wrong class
    return probabilities[predicted_class]

print("Utility functions defined successfully")

✓ Utility functions defined successfully
